In [1]:
import numpy as np
import os
import pandas as pd 

def create_cue_dynam(highProb=0.8, lowProb=0.2, neutral=1.0, trials_per_cue=10):
    cue_data = {"cue_names": ["Sea animal",  "Water vehicle",  "Neutral"],
                "cue_letters": ["SA", "WV", "X"],
                "cue_color" : [[0.4, 0.6, 1.0], [-0.4, -0.2, 0.4], [-1, -1, -1]],
                "cue_highProb_cats": [["dolphin", "whale"], 
                                    ["speedboat", "submarine"], 
                                    ["dolphin", "whale", "speedboat", "submarine"]],
                
                "cue_lowProb_cats": [["speedboat", "submarine"], 
                                    ["dolphin", "whale"],
                                    ["dolphin", "whale", "speedboat", "submarine"]]}

    cue_data["cue_highProb"] = []
    cue_data["cue_lowProb"] = []
    
    for cue_id, cue in enumerate(cue_data["cue_names"]):
        if cue != "Neutral":
            cue_data["cue_highProb"].append([np.round(highProb / len(cue_data["cue_highProb_cats"][cue_id]), 2)] * len(cue_data["cue_highProb_cats"][cue_id]))
            cue_data["cue_lowProb"].append([np.round(lowProb / len(cue_data["cue_lowProb_cats"][cue_id]), 2)] * len(cue_data["cue_lowProb_cats"][cue_id]))
        
        else:
            cue_data["cue_highProb"].append([np.round(neutral / len(cue_data["cue_highProb_cats"][cue_id]), 3)] * len(cue_data["cue_highProb_cats"][cue_id]))
            cue_data["cue_lowProb"].append([np.round(neutral / len(cue_data["cue_lowProb_cats"][cue_id]), 3)] * len(cue_data["cue_highProb_cats"][cue_id]))
            
    cue_data["high_prob_trials"] = [np.array(x) * trials_per_cue for x in cue_data["cue_highProb"]] 
    cue_data["low_prob_trials"] =  [np.array(x) * trials_per_cue for x in cue_data["cue_lowProb"]]
    
    return cue_data
        

In [6]:
stim_path = "/projects/crunchie/boyanova/EEG_Things/Mask_ExpAtt_EEG/stimuli"

def create_block_trials(stim_path, cue_data): 
    categories = os.listdir(stim_path)
    stimuli = []
    for cat in categories:
        cat_path = os.path.join(stim_path, cat)
        files = os.listdir(cat_path)
        stimuli.extend([f".\\stimuli\\{cat}\\{x}" for x in files])

    stimuli = np.array(stimuli)[np.argsort(stimuli)]
    data = {"target_id": [],
            "distractor_id": [],
            "target": [],
            "distractor": [],
            "expectation": [],
            "mask_ISI": [],
            "cue": [],
            "cue_letter": [],
            "target_name": [],
            "target_cat": [],}

    mask_type = [0.017, 0.1]
    distractors = stimuli[np.char.count(stimuli, "mask") > 0]
    distractor_ids = np.arange(len(stimuli))[np.isin(stimuli, distractors)]
    local_distractor_ids = np.arange(0, len(distractors))

    for cue_id, cue in enumerate(cue_data["cue_names"]):
        high_cats = np.array(cue_data["cue_highProb_cats"][cue_id])
        low_cats = np.array(cue_data["cue_lowProb_cats"][cue_id])
        
        if cue != "Neutral":
            for i, l_cat in enumerate(low_cats):
                l_cat_stim = stimuli[np.char.count(stimuli, l_cat) > 0]
                targets = l_cat_stim[np.char.count(l_cat_stim, "mask") == 0]
                target_ids = np.arange(len(stimuli))[np.isin(stimuli, targets)]
            

                trial_collectos = []
                for mask in mask_type:
                
                    h_trials = cue_data["low_prob_trials"][cue_id][i]
                    split_h_trials = int(h_trials // 2)
                    data["target_id"].extend(np.repeat(target_ids , split_h_trials))
                    data["target"].extend(np.repeat(targets, split_h_trials))
                    
                    target_names =[x.split("\\")[-1] for x in targets]
                    target_categories = [x.split("_")[0] for x in target_names]
                    
                    random_distractors = np.random.choice(local_distractor_ids, split_h_trials * 2)
                    data["distractor_id"].extend([distractor_ids[x] for x in random_distractors])
                    data["distractor"].extend([distractors[x] for x in random_distractors])
                    data["target_name"].extend(np.repeat(target_names , split_h_trials))
                    data["target_cat"].extend(np.repeat(target_categories , split_h_trials))
                    
                    if cue != "Neutral":
                        data["expectation"].extend(["unexpected"] * split_h_trials * 2)
                    else:
                        data["expectation"].extend(["neutral"]* split_h_trials * 2)
                        
                    data["mask_ISI"].extend([mask] * split_h_trials * 2)
                    data["cue"].extend([cue] * split_h_trials * 2)
                    data["cue_letter"].extend([cue_data["cue_letters"][cue_id]]* split_h_trials * 2)
                    
        for i, h_cat in enumerate(high_cats):
            h_cat_stim = stimuli[np.char.count(stimuli, h_cat) > 0]
            targets = h_cat_stim[np.char.count(h_cat_stim, "mask") == 0]
            target_ids = np.arange(len(stimuli))[np.isin(stimuli, targets)]

            trial_collectos = []
            for mask in mask_type:
                h_trials = cue_data["high_prob_trials"][cue_id][i]
                split_h_trials = int(h_trials // 2)
                
                data["target_id"].extend(np.repeat(target_ids , split_h_trials))
                data["target"].extend(np.repeat(targets , split_h_trials))
                
                target_names =[x.split("\\")[-1] for x in targets]
                target_categories = [x.split("_")[0] for x in target_names]
                
                random_distractors = np.random.choice(local_distractor_ids, split_h_trials * 2)
                data["distractor_id"].extend([distractor_ids[x] for x in random_distractors])
                data["distractor"].extend([distractors[x] for x in random_distractors])
                data["target_name"].extend(np.repeat(target_names , split_h_trials))
                data["target_cat"].extend(np.repeat(target_categories , split_h_trials))
                
                
                if cue != "Neutral":
                    data["expectation"].extend(["expected"] * split_h_trials * 2)
                else:
                    data["expectation"].extend(["neutral"]* split_h_trials * 2)
                    
                data["mask_ISI"].extend([mask] * split_h_trials * 2)
                data["cue"].extend([cue] * split_h_trials * 2)
                data["cue_letter"].extend([cue_data["cue_letters"][cue_id]] * split_h_trials * 2)
                

                
    data["target_loc"] = [np.random.choice(["L", "R"], 1, p=[0.5, 0.5])[0] for _ in range(len(data["target"]))]
    data["distractor_loc"] = ["R" if x == "L" else "L" for x in data["target_loc"]]
    df = pd.DataFrame(data)
    df = df.sample(frac=1).reset_index(drop=True)
    return df
    

In [2]:
cue_data = create_cue_dynam()

In [7]:
cue_data = create_cue_dynam()
stim_path = "/projects/crunchie/boyanova/EEG_Things/Mask_ExpAtt_EEG/stimuli"
trials = create_block_trials(stim_path, cue_data)

In [11]:
trials[trials.target_id == 1]

,target_id,distractor_id,target,distractor,expectation,mask_ISI,cue,cue_letter,target_name,target_cat,target_loc,distractor_loc
6,1,12,.\stimuli\dolphin\dolphin_01.jpg,.\stimuli\whale\mask-whale_00.png,expected,0.100,Sea animal,SA,dolphin_01.jpg,dolphin,R,L
11,1,4,.\stimuli\dolphin\dolphin_01.jpg,.\stimuli\speedboat\mask-speedboat_00.png,expected,0.017,Sea animal,SA,dolphin_01.jpg,dolphin,R,L
12,1,4,.\stimuli\dolphin\dolphin_01.jpg,.\stimuli\speedboat\mask-speedboat_00.png,neutral,0.100,Neutral,X,dolphin_01.jpg,dolphin,R,L
25,1,8,.\stimuli\dolphin\dolphin_01.jpg,.\stimuli\submarine\mask-submarine_00.png,neutral,0.017,Neutral,X,dolphin_01.jpg,dolphin,L,R
37,1,2,.\stimuli\dolphin\dolphin_01.jpg,.\stimuli\dolphin\mask-dolphin_00.png,expected,0.100,Sea animal,SA,dolphin_01.jpg,dolphin,R,L
45,1,12,.\stimuli\dolphin\dolphin_01.jpg,.\stimuli\whale\mask-whale_00.png,expected,0.017,Sea animal,SA,dolphin_01.jpg,dolphin,L,R


In [19]:
trials[(trials.target_id == 0)]

,target_id,distractor_id,target,distractor,expectation,mask_ISI,cue,cue_letter,target_name,target_cat,target_side,distractor_side
24,0,3,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\dolphin\mask-dolphin_01.png,expected,0.017,Sea animal,SA,dolphin_00.jpg,dolphin,L,R
25,0,2,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\dolphin\mask-dolphin_00.png,expected,0.017,Sea animal,SA,dolphin_00.jpg,dolphin,R,L
26,0,12,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\whale\mask-whale_00.png,expected,0.017,Sea animal,SA,dolphin_00.jpg,dolphin,R,L
27,0,4,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\speedboat\mask-speedboat_00.png,expected,0.017,Sea animal,SA,dolphin_00.jpg,dolphin,R,L
28,0,3,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\dolphin\mask-dolphin_01.png,expected,0.017,Sea animal,SA,dolphin_00.jpg,dolphin,L,R
29,0,3,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\dolphin\mask-dolphin_01.png,expected,0.017,Sea animal,SA,dolphin_00.jpg,dolphin,R,L
30,0,3,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\dolphin\mask-dolphin_01.png,expected,0.017,Sea animal,SA,dolphin_00.jpg,dolphin,L,R
38,0,12,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\whale\mask-whale_00.png,expected,0.100,Sea animal,SA,dolphin_00.jpg,dolphin,R,L
39,0,8,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\submarine\mask-submarine_00.png,expected,0.100,Sea animal,SA,dolphin_00.jpg,dolphin,R,L
40,0,8,.\_stimuli\dolphin\dolphin_00.jpg,.\_stimuli\submarine\mask-submarine_00.png,expected,0.100,Sea animal,SA,dolphin_00.jpg,dolphin,R,L


In [1]:
import os
from PIL import Image

root_dir = "/projects/crunchie/boyanova/EEG_Things/Mask_ExpAtt_EEG/target_stimuli"
target_size = (500, 500)

# Image extensions to process
valid_exts = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp")

for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.lower().endswith(valid_exts):
            img_path = os.path.join(root, file)

            try:
                with Image.open(img_path) as img:
                    img = img.convert("RGB")  # safe for consistency
                    img_resized = img.resize(target_size, Image.LANCZOS)
                    img_resized.save(img_path)

            except Exception as e:
                print(f"Failed to process {img_path}: {e}")


In [15]:
import os 
import numpy as np

picks = ["dolphin", "submarine", "hawk", "jet", "helicopter"]
target_names = []
distractor_names = []
all_categories = np.array(os.listdir("./target_stimuli"))
options = []
for t_ct in all_categories:
    if t_ct in picks:
        t_ct_pt = os.path.join("./target_stimuli", t_ct)
        t_files = os.listdir(t_ct_pt)
        m_ct = np.random.choice(all_categories[~np.isin(all_categories, t_ct)], size=1)[0]
        m_ct_pt = os.path.join("./distractor_stimuli", m_ct)
        m_files = os.listdir(m_ct_pt)
        target_names.append(f".\\target_stimuli\\{t_ct}\\{np.random.choice(t_files, size=1)[0]}")
        distractor_names.append(f".\\distractor_stimuli\\{m_ct}\\{np.random.choice(m_files, size=1)[0]}")
        


In [19]:
1000/60.05

16.65278934221482

In [ ]:
import os 
general_img_duration = 1000/60.05 ## requeres global clock to assure no additional frames! Monirot refresh rate is 60.05Hz 

image_data = {"target_image": ['.\\target_stimuli\\dolphin\\dolphin_14s.jpg',
                                '.\\target_stimuli\\hawk\\hawk_10s.jpg',
                                '.\\target_stimuli\\helicopter\\helicopter_06s.jpg',
                                '.\\target_stimuli\\jet\\jet_10s.jpg',
                                '.\\target_stimuli\\submarine\\submarine_12s.jpg'],
              "distractor_image": ['.\\distractor_stimuli\\submarine\\mask-submarine_12s.png',
                                '.\\distractor_stimuli\\dolphin\\mask-dolphin_14s.png',
                                '.\\distractor_stimuli\\whale\\mask-whale_12s.png',
                                '.\\distractor_stimuli\\eagle\\mask-eagle_15s.png',
                                '.\\distractor_stimuli\\helicopter\\mask-helicopter_25s.png']
              "image_mask_isi": [1000/60.05, 1000/60.05, 300, 300, 300]
              "target_loc": ["L", "R", "L", "R", "L"]
              "distractor_loc": ["R", "L", "R", "L", "R"] }

masks_paths = "C:\\Users\\bozanova\\Desktop\\things_mask\\masks" 
all_masks = os.listdir(masks_paths)

###
#...needs to go somewhere in the loop 
left_masks = np.random.choice(all_masks, 12, replace=False)
right_masks = np.random.choice(all_masks, 12, replace=False)
left_masks_paths = [os.path.join(masks_paths, x) for x in left_masks]
right_masks_paths = [os.path.join(masks_paths, x) for x in right_masks]

